In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "Qwen/Qwen2.5-0.5B" #path/to/your/model/or/name/on/hub"
adapter_model_name = "opt"
device = "cuda" # or "cuda" if you have a GPU

model = AutoModelForCausalLM.from_pretrained(base_model_name).to(device)
model = PeftModel.from_pretrained(model, adapter_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

/home/esdfirls/miniconda3/envs/supportiv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [ ]:
inputs = tokenizer.encode("What is glaucoma?", return_tensors="pt").to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

In [17]:
# Initialize the DataProcessor with the path to the MLE screening dataset
from data_processing import DataProcessor


dp = DataProcessor("files/mle_screening_dataset.csv")    

# Load the datasets
mle_set = dp.load_data_screening()
# pub_set = dp.load_data_pubmedqa()

# Concatenate datasets
# dataset = dp.concatenate_datasets(mle_set, pub_set)

# Split the dataset into training and validation sets
train, validation, test = dp.train_test_validation_split(mle_set)
# Add an 'id' column to each dataset
train = train.map(lambda example, idx: {**example, "id": idx}, with_indices=True)
validation = validation.map(lambda example, idx: {**example, "id": idx}, with_indices=True)
test = test.map(lambda example, idx: {**example, "id": idx}, with_indices=True)
# train, validation, test = dp.transform_dataset(train), dp.transform_dataset(validation), dp.transform_dataset(test)

Map: 100%|██████████| 1641/1641 [00:00<00:00, 17995.89 examples/s]


In [14]:
chat = [
    {"role": "system", "content": "You are helpful"},
    {"role": "user", "content": "What is (are) Surviving Cancer ?"},
]

import torch
from transformers import pipeline

pipeline = pipeline(task="text-generation", model="qwen_0.5_mle/", torch_dtype=torch.bfloat16, device_map="auto", tokenizer=tokenizer)
response = pipeline(chat, max_new_tokens=512)


Device set to use cuda:0


In [15]:
response

[{'generated_text': [{'role': 'system', 'content': 'You are helpful'},
   {'role': 'user', 'content': 'What is (are) Surviving Cancer ?'},
   {'role': 'assistant',
    'content': 'Surviving cancer is the ability to survive the disease. Surviving cancer is a very important goal for many people. It is also a goal for many cancer survivors. Surviving cancer means that the cancer has not spread to other parts of the body. It means that the cancer has not spread to the brain, bones, or other parts of the body. It means that the cancer has not spread to the lymph nodes, which are small, bean-shaped structures that are found throughout the body. It means that the cancer has not spread to the liver, spleen, or other organs. It means that the cancer has not spread to the lungs, bones, or other parts of the body. It means that the cancer has not spread to the brain, bones, or other parts of the body. It means that the cancer has not spread to the liver, spleen, or other organs. It means that the

In [ ]:
from evaluate_model import evaluator

task_evaluator = evaluator("text-generation")
results = task_evaluator.compute(
    model_or_pipeline=pipeline,
    data=validation,
    metric=["f1", "rouge", "bleu"],
    input_column="question",
    label_column="answer",
)
print(results)